# Landsat Workflow 

The Landsat harvesting workflow consists of two BPMN processes. The main process (Landsat Registration) searches for new data at USGS. For each new scene found, the workflow executes another process (Landsat Scene Ingestion) which performs the individual steps for harvesting and registering the data.

This notebook demonstates the deployment of both processes at the Flowable instance in the EOEPCA Develop-Cluster and checks the next execution time of the main process.

In [ ]:
from requests import Session
from requests.auth import HTTPBasicAuth
import json

flowable_base_url = "https://registration-harvester-api.develop.eoepca.org/flowable-rest"
flowable_rest_user = "eoepca"
flowable_rest_pw = "eoepca"
bpmn_landsat_registration = "../workflows/landsat.bpmn"
bpmn_landsat_scene_ingestion = "../workflows/landsat-scene-ingestion.bpmn"

flowable_session = Session()
flowable_session.auth = HTTPBasicAuth(flowable_rest_user, flowable_rest_pw)

## Deploy the BPMN processes

First, deploy the main process.

In [ ]:
bpmn_file = {"file": open(bpmn_landsat_registration, "rb")}
response = flowable_session.post(f"{flowable_base_url}/service/repository/deployments", files=bpmn_file)
print(json.dumps(response.json(), indent=4))

Then deploy the subprocess which implements the harvesting and registration of each scene discovered by the main process.

In [ ]:
bpmn_file = {"file": open(bpmn_landsat_scene_ingestion, "rb")}
response = flowable_session.post(f"{flowable_base_url}/service/repository/deployments", files=bpmn_file)
print(json.dumps(response.json(), indent=4))

## Check processes and workflow execution timer

Both processes, named `Landsat Registration` and `Landsat Scene Ingestion` are now available at the Flowable instance.

In [ ]:
response = flowable_session.get(f"{flowable_base_url}/service/repository/process-definitions")
processes = response.json()['data']
for process in processes:
    print(f"Process Definition: name='{process['name']}' id='{process['id']}'")

The main process (Landsat Registration) is supposed to start automatically. A corresponding timer is started when the process is deployed. The datetime of the next workflow execution can be checked. The process id of the timer matches the id of the Landsat Registration process.

In [ ]:
response = flowable_session.get(f"{flowable_base_url}/service/management/timer-jobs")
timer_jobs = response.json()['data']
for job in timer_jobs:
    print(f"Timer for process with id {job['processDefinitionId']} avaiable. Next execution at {job['dueDate']}.")